# 패키지 설치     
 - Audio processing : librosa, mutagen
 - Plots : Plotly, matplotlib

In [2]:
#패키지 설치
!pip install pandas
!pip install librosa
!pip install plotly
!pip install matplotlib
!pip install mutagen
!pip install pillow

In [3]:
import os
import time
import librosa
import zipfile
import mutagen
import mutagen.wave
import numpy as np
import pandas as pd
import librosa.display
import IPython.display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image

#전이학습에 있는 모듈 임포트
import glob
from matplotlib.pyplot import specgram
import soundfile as sf

# 데이터셋 불러오기     
 - Urbansound8K

In [4]:
#실행되는 디렉토리 수정
# %cd drive/MyDrive/(Colab\Notebooks)/폴더명
%cd C:\Users\SCLAB\USC\AudioData\UrbanSound8K

C:\Users\SCLAB\USC\AudioData\UrbanSound8K


# 데이터 분석

In [5]:
#함수정의
def create_dataset_df(csv_file):
    dataset_df = pd.read_csv(csv_file)
    filepaths = []
    for i, row in dataset_df.iterrows():
        filepaths.append(os.path.join('UrbanSound8K/audio', 'fold'+str(row['fold']), row['slice_file_name']))
    dataset_df['filepath'] = filepaths
    return dataset_df

In [6]:
#5개의 데이터셋 확인하기
dataset_df = create_dataset_df('/Users/SCLAB/USC/AudioData/UrbanSound8K/metadata/UrbanSound8K.csv')
dataset_df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,filepath
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,UrbanSound8K/audio\fold5\100032-3-0-0.wav
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,UrbanSound8K/audio\fold5\100263-2-0-117.wav
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,UrbanSound8K/audio\fold5\100263-2-0-121.wav
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,UrbanSound8K/audio\fold5\100263-2-0-126.wav
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,UrbanSound8K/audio\fold5\100263-2-0-137.wav


In [7]:
#클래스별 데이터 개수 확인하기
dataset_df.groupby('class').slice_file_name.count()

class
air_conditioner     1000
car_horn             429
children_playing    1000
dog_bark            1000
drilling            1000
engine_idling       1000
gun_shot             374
jackhammer          1000
siren                929
street_music        1000
Name: slice_file_name, dtype: int64

파이썬에서 데이터 분석, 처리를 할 때 Pandas를 사용     
groupby()는 같은 값을 하나로 묶어 통계 또는 집계 결과를 얻기 위해 사용

# Audio Set 불러오기   
 1. 복제 리포지토리 설정

In [10]:
#깃 설치하기
!conda install git

Solving environment: ...working... done

# All requested packages already installed.



In [7]:
#깃허브 주소에 있는 파일을 컴퓨터로 다운로드
#리포지토리의 원주소인 url을 복사하고 git clone 명령어 실행
!git clone https://github.com/aoifemcdonagh/audioset-processing.git

fatal: destination path 'audioset-processing' already exists and is not an empty directory.


In [8]:
#실행되고 있는 디렉토리를 변경
%cd audioset-processing/

C:\Users\SCLAB\Alarmi\audioset-processing


2. 요구 사항 설치   
 - 런타임을 다시 시작하라는 메세지 무시

In [20]:
!pip install -r requirements.txt

  Created wheel for pefile: filename=pefile-2019.4.18-py3-none-any.whl size=60821 sha256=991858322cb8a51aaacc51a39aef6e6603369cab82502cc8bae8d40ba92c02ab
  Stored in directory: c:\users\sclab\appdata\local\pip\cache\wheels\42\52\d5\9550bbfb9eeceaf0f19db1cf651cc8ba41d3bcf8b4d20e4279
Successfully built pefile


3. AudioSet에서 클래스의 모든 인스턴스 다운로드
 - 아나콘다 4.12.0
 - 파이썬 3.8.3

In [1]:
#기침(871개)
!python3 process.py download -c "cough"

Python


In [1]:
!cd audioset-processing

In [11]:
!pwd

/c/Users/SCLAB/Alarmi


In [10]:
# Processing utility for AudioSet dataset
# Sort, find and download the entire, or subsets of, AudioSet

# Aoife McDonagh

import argparse
import os
import .core.utils as utils
#from core.utils import *


def find(args):
    """
    Function for finding all examples in a directory containing labels for given classes
    :param args:
    :return:
    """
    print("Finding all files labeled with classes" + args.classes + " in " + args.audio_data_dir)

    for class_name in args.classes:
        utils.find(class_name, args)
        print("Finished finding and sorting files for class: " + class_name)


def download(args):
    """
    Function for downloading all examples in AudioSet containing labels for given classes
    :param args:
    :return:
    """
    print("Downloading classes from AudioSet.")

    for class_name in args.classes:
        utils.download(class_name, args)



parser = argparse.ArgumentParser()
parser.add_argument('mode', type=str, choices=['find', 'download'])
parser.add_argument('-c', '--classes', nargs='+', type=str,
                    help='list of classes to find in a given directory of audioset files')
parser.add_argument('-b', '--blacklist', nargs='+', type=str,
                    help='list of classes which will exclude a clip from being downloaded')
parser.add_argument('-d', '--destination_dir', type=str,
                    help='directory path to put downloaded (or found) files into')
parser.add_argument('--audio_data_dir', type=str,
                    help='directory path containing pre-downloaded files from AudioSet')
parser.add_argument('-fs', "--sample_rate", type=int,
                    help="Sample rate of audio to download. Default 16kHz")
parser.add_argument('-s', '--strict',
                    help='If used, only match exact string argument passed')
parser.add_argument('--label_file', type=str,
                    help='Path to CSV file containing AudioSet labels for each class')
parser.add_argument('--csv_dataset', type=str,
                    help='Path to CSV file containing AudioSet in YouTube-id/timestamp form')

parser.set_defaults(
    label_file='./data/class_labels_indices.csv',
    csv_dataset='./data/unbalanced_train_segments.csv',
    destination_dir='./output',
    fs=16000
)

args = parser.parse_args()

if args.mode == 'find':
    if args.destination_dir is not None and not os.path.isdir(args.destination_dir):
        os.makedirs(args.destination_dir)
    find(args)

elif args.mode == 'download':
    if args.destination_dir is not None and not os.path.isdir(args.destination_dir):
        os.makedirs(args.destination_dir)
    download(args)




SyntaxError: invalid syntax (<ipython-input-10-8667ac229b6f>, line 8)